In [ ]:
# запускаем в colab или локально?
try:
    from google.colab import drive
    colab = True
except ImportError:
    colab = False

print(f"colab: {colab}")

In [ ]:
# установка необходимых пакетов в colab
if colab:
    !pip install rootutils -q
    !pip install torchinfo -q
    !pip install torchmetrics -q
    !pip install livelossplot -q

In [ ]:
# монтирование google диска и установка
# рабочей директории в `computer-vision`

import os
from rootutils import setup_root

if colab:
    drive.mount("/content/drive", force_remount=True)
    os.chdir("drive/MyDrive/computer-vision")
    root = setup_root(".", indicator="homeworks", pythonpath=True)
else:
    root = setup_root(".", indicator="homeworks", pythonpath=True)

os.chdir(root)
print(f"working directory: {os.getcwd()}")


In [ ]:
# создание директории для данных

from pathlib import Path

if colab:
    DATA_DIR = Path("/content/data")
else:
    DATA_DIR = root / "data"

DATA_DIR.mkdir(exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")

In [ ]:
# настройка matplotlib

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format="retina"

plt.style.use("seaborn-v0_8-notebook")

## **Операция свертки. Сверточные нейронные сети**

 <center>
    <figure>
        <img src="../figures/03/invariant.jpg" width="600px"/>
    </figure>
</center>

- Нейронная сеть называется **инвариантной** к преобразованию $\mathbf{t}[\mathbf{x}]$ изображения, если:
    $$
        \mathbf{f}\left[\mathbf{t}[\mathbf{x}]\right] = 
        \mathbf{f}[\mathbf{x}]
    $$
    Например, классификатор изображений должен быть инвариантен к сдвигу изображений.

- Нейронная сеть называется **ковариантной** к преобразованию $\mathbf{t}[\mathbf{x}]$ изображения, если:
    $$
        \mathbf{f}\left[\mathbf{t}[\mathbf{x}]\right] = 
        \mathbf{t}\left[\mathbf{f}[\mathbf{x}]\right]
    $$
    Например, сегментация изображений должна быть ковариантна к сдвигу изображений.

- Полносвязные нейронные сети не инварианты и не ковариантны к сдвигу изображений.

- Сверточные сети состоят из последовательности сверточных слоев, каждый из которых ковариантен для переноса. 

- Сверточные сети также обычно включают операции pooling, которые способствуют частичной инвариантности к переносу.

### **1. Одномерная операция свертки**

Строго говоря, то что мы будем обсуждать, это корреляция, а не свертка. 

Как бы то ни было, это (некорректное) определение - общепринятое соглашение в машинном обучении.

#### **1.1 Примеры одномерных сверток**

**(a) Пример cвертки одномерного вектора $\mathbf{x}$ c:**

- size = 3 (размер ядра свертки)

- stride = 1 (шаг)
- dilation = 1 (расширение)
- no padding (без дополнения нулями)


<center>
    <figure>
        <img src="../figures/03/conv1d_1.jpg" width="400px"/>
    </figure>
</center>

Формула для свертки:
$$
    z_i = \omega_1 x_{i-1} + \omega_2 x_{i} + \omega_3 x_{i+1}
$$

где $\omega_1\,,  \omega_2\,, \omega_3$ - ядро свертки (веса свертки).

Видно, что свертка - ковариантная относительно сдвига операция.


**(b) Пример cвертки одномерного вектора $\mathbf{x}$ с:**

- size = 3 (размер ядра свертки)

- stride = 1 (шаг)
- dilation = 1 (расширение)
- zero padding (c дополнением нулями)

<center>
    <figure>
        <img src="../figures/03/conv1d_2.jpg" width="180px"/>
    </figure>
</center>

С zero padding размер входного и выходного вектора совпадают.

**(c) Еще примеры одномерных сверток:**

<center>
    <figure>
        <img src="../figures/03/conv1d_3.jpg" width="800px"/>
    </figure>
</center>

В расширенных (dilated) свертках мы прореживаем вектор весов нулями, что позволит комбинировать информацию с больших участков, используя меньше весов.

#### **1.2 Размер выхода свертки**

Размер выходного вектора $\mathbf{z}$ после свертки, примерно в stride раз меньше размера входного вектора $\mathbf{x}$:
$$
    o = \left\lfloor\frac{n + 2p - k}{s}\right\rfloor
$$
где 
- $o$ - размер $\mathbf{z}$

- $n$ - размер $\mathbf{x}$
- $k$ - размер ядра свертки
- $s$ - шаг (stride) свертки
- $p$ - число нулей с каждой стороны входного вектора

Эти формулы можно распространить на произвольное количество измерений (в двумерных и трехмерных свертках), повторив их для каждого из них.

#### **1.3 Сверточные слои**

- С размером ядра 3, шагом 1 и расширением (dilation) 1, $i$-й скрытый элемент $h_i$ сверточного слоя вычисляется как:
    $$
        h_i = a\left[\beta + \omega_1 x_{i-1} + \omega_2 x_{i} + \omega_3 x_{i+1}\right]
    $$
    где смещение $\beta$ и веса ядра $\omega_1$, $\omega_2$, $\omega_3$ - параметры, которые мы хотим обучить.

- Это специальный случай полносвязного слоя:
$$
    h_i = a\left[\beta_i + \sum_{j=1}^D\omega_{ij}x_j\right]
$$

- В полносвязном слое намного больше параметров: для $D$ входов и $D$ скрытых элементов в полносвязном слое будет $D^2 + D$ параметров.

**Сравнение числа параметров в полносвязном слое и в свертках с ядром 3 и шагами 1 и 2:**

<center>
    <figure>
        <img src="../figures/03/weights.jpg" width="800px"/>
    </figure>
</center>

#### **1.4 Каналы**

- Обычно вычисляют несколько сверток параллельно. 
  
  Каждая свертка дает новый набор скрытых переменных, который называется **картой признаков** или **каналом**:

<center>
    <figure>
        <img src="../figures/03/channels_1.jpg" width="500px"/>
    </figure>
</center>

- В общем случае вход и все скрытые слои имеют по несколько каналов.

  Если входной слой имеет $C_i$ каналов, в следующем слое $C_o$ каналов и ядро размера $K$, то нужно:
  $$
    C_i \cdot C_o \cdot K \quad \text{- весов и}
  $$
  $$
    C_o \quad \text{- смещений}
  $$

<center>
    <figure>
        <img src="../figures/03/channels_2.jpg" width="300px"/>
    </figure>
</center>


#### **1.5 Рецептивные поля**

<center>
    <figure>
        <img src="../figures/03/fields.jpg" width="600px"/>
    </figure>
</center>

C увеличением числа скрытых слоев, рецептивные поля скрытых элементов увеличиваются.

<center>
    <figure>
        <img src="../figures/03/features.jpg" width="600px"/>
    </figure>
</center>

Увеличение рецептивных полей, в случае двумерной свертки приводит к точу, что каждый следующий скрытый слой
выучивает более высокоуровневые признаки на изображении.

#### **1.6 PyTorch code for 1D edge detection**

In [ ]:
import torch

x = torch.tensor(
    [10, 11, 9, 10, 101, 99, 100, 101, 9, 10, 11, 10],
    dtype=torch.float32
)

print(f"x.shape: {x.shape}")

plt.figure(figsize=(6, 3))
plt.plot(x, "bo");
plt.xlabel("i")
plt.ylabel("x_i")

In [ ]:
# ядро свертки (вычисление производной)
w = torch.tensor([0.5, -0.5])

# преобразование размера к 1 x 1 x 12 
# (batch size) x (num channels) x (len of x)
x = x.unsqueeze(0).unsqueeze(0) 

# преобразование размера к 1 x 1 x 2
w = w.unsqueeze(0).unsqueeze(0)

# операция 1D свертки
conv1d = torch.nn.Conv1d(1, 1, kernel_size=3, stride=1, padding=0, dilation=1, bias=False)

# зададим веса свертки
conv1d.weight = torch.nn.Parameter(w, requires_grad=False) 

with torch.inference_mode():
    y = conv1d(x) 

print(f"y.shape: {y.shape}")

# 1 x 1 x 11 -> [11]
y = y.flatten()

plt.figure(figsize=(6, 3))
plt.plot(y, "ro");
plt.xlabel("i")
plt.ylabel("y_i")